### Query MySQL for data

In [1]:
import pandas as pd
import network_vis_helper as visnet
import api_wrapper as api

## reading in data until the database is established
user_info = pd.read_csv('data/all_users.csv')
activity = pd.read_csv('data/activity.csv')
followed_accounts = pd.read_csv('data/followed_accounts.csv')
el = pd.read_csv('data/el.csv')

### Visualize networks

In [2]:
weighted_el = visnet.get_weighted_el(el)
visnet.render_graph(weighted_el, edge_type = 'mention', edge_weight_threshold = 3)

### Who are the most mentioned users for each seed?

In [3]:
source_list = weighted_el['source'].unique().tolist()
for source in source_list:
    print(weighted_el[['source', 'target', 'weight']][(weighted_el['source'] == source) & 
                                                      (weighted_el['edge_type'] == 'mention') &
                                                      (weighted_el['weight'] >= 5) ]
            .sort_values('weight', ascending = False))
    print("\n")

         source           target  weight
79  AMJAMAMJAM1  umdhealthequity       5


           source      target  weight
113  Briony_Swire  davidlazer      11
108  Briony_Swire   UlliEcker       7


           source           target  weight
165  Broniatowski           GWIDDP      19
329  Broniatowski      noelTbrewer      12
204  Broniatowski        adamgdunn       9
171  Broniatowski    GordPennycook       8
319  Broniatowski      jayvanbavel       8
150  Broniatowski     Broniatowski       7
158  Broniatowski          DG_Rand       7
197  Broniatowski  RebekahKTromble       7
142  Broniatowski      AMJAMAMJAM1       6
143  Broniatowski  AMJPublicHealth       6
318  Broniatowski       j_a_tucker       6
163  Broniatowski       EHarrisNYU       5
333  Broniatowski  philipparnamets       5
342  Broniatowski     steverathje2       5


      source           target  weight
379  DG_Rand    GordPennycook      33
436  DG_Rand        _mohsen_m      16
461  DG_Rand       deaneckles      11
3